# RAG pipeline using Optimized and Quantized Embedders

In this tutorial, we will demo how to build a RAG pipeline, with the embedding for all documents done using Quantized Embedders.

We will use a pipeline that will:

* Create a document collection.
* Embed all documents using Quantized Embedders.
* Fetch relevant documents for our question.
* Run an LLM answer the question.

For more information about optimized models, we refer to [optimum-intel](https://github.com/huggingface/optimum-intel.git) and [IPEX](https://github.com/intel/intel-extension-for-pytorch).

This tutorial is based on the [Langchain RAG tutorial here](https://towardsai.net/p/machine-learning/dense-x-retrieval-technique-in-langchain-and-llamaindex) and [Langchain cookbook here] (https://github.com/langchain-ai/langchain/blob/master/cookbook/rag_with_quantized_embeddings.ipynb).

In [27]:
import uuid
from pathlib import Path

import langchain
import torch
from bs4 import BeautifulSoup as Soup
from langchain.retrievers.multi_vector import MultiVectorRetriever
from langchain.storage import InMemoryByteStore, LocalFileStore
from langchain_community.document_loaders.recursive_url_loader import (
    RecursiveUrlLoader,
)
from langchain_community.vectorstores import Chroma

# For our example, we'll load docs from the web
from langchain_text_splitters import RecursiveCharacterTextSplitter

DOCSTORE_DIR = "."
DOCSTORE_ID_KEY = "doc_id"

Lets first load up this paper, and split into text chunks of size 1000.

In [28]:
# Could add more parsing here, as it's very raw.
loader = RecursiveUrlLoader(
    "https://ar5iv.labs.arxiv.org/html/1706.03762",
    max_depth=2,
    extractor=lambda x: Soup(x, "html.parser").text,
)
data = loader.load()
print(f"Loaded {len(data)} documents")

# Split
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
all_splits = text_splitter.split_documents(data)
print(f"Split into {len(all_splits)} documents")

Loaded 1 documents
Split into 53 documents


In order to embed our documents, we can use the ```QuantizedBiEncoderEmbeddings```, for efficient and fast embedding. 

In [29]:
from langchain_community.embeddings import QuantizedBiEncoderEmbeddings
from langchain_core.embeddings import Embeddings

model_name = "Intel/bge-small-en-v1.5-rag-int8-static"
encode_kwargs = {"normalize_embeddings": True}  # set True to compute cosine similarity

model_inc = QuantizedBiEncoderEmbeddings(
    model_name=model_name,
    encode_kwargs=encode_kwargs,
    query_instruction="Represent this sentence for searching relevant passages: ",
)

/home/rbrugaro/miniforge3/envs/llama-index/lib/python3.10/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
Passing the argument `library_name` to `get_supported_tasks_for_model_type` is required, but got library_name=None. Defaulting to `transformers`. An error will be raised in a future version of Optimum if `library_name` is not provided.
/home/rbrugaro/miniforge3/envs/llama-index/lib/python3.10/site-packages/torch/amp/autocast_mode.py:267: UserWarning: In CPU autocast, but the target dtype is not supported. Disabling autocast.
CPU Autocast only supports dtype of torch.bfloat16, torch.float16 currently.
  warnings.warn(error_message)


With our embedder in place, lets define our retriever:

In [30]:
def get_multi_vector_retriever(
    docstore_id_key: str, collection_name: str, embedding_function: Embeddings
):
    """Create the composed retriever object."""
    vectorstore = Chroma(
        collection_name=collection_name,
        embedding_function=embedding_function,
    )
    store = InMemoryByteStore()

    return MultiVectorRetriever(
        vectorstore=vectorstore,
        byte_store=store,
        id_key=docstore_id_key,
    )


retriever = get_multi_vector_retriever(DOCSTORE_ID_KEY, "multi_vec_store", model_inc)

Next, we divide each chunk into sub-docs:

In [31]:
child_text_splitter = RecursiveCharacterTextSplitter(chunk_size=400)
id_key = "doc_id"
doc_ids = [str(uuid.uuid4()) for _ in all_splits]

In [32]:
sub_docs = []
for i, doc in enumerate(all_splits):
    _id = doc_ids[i]
    _sub_docs = child_text_splitter.split_documents([doc])
    for _doc in _sub_docs:
        _doc.metadata[id_key] = _id
    sub_docs.extend(_sub_docs)

Lets write our documents into our new store. This will use our embedder on each document.

In [33]:
retriever.vectorstore.add_documents(sub_docs)
retriever.docstore.mset(list(zip(doc_ids, all_splits)))

Batches:   0%|          | 0/6 [00:00<?, ?it/s]

/home/rbrugaro/miniforge3/envs/llama-index/lib/python3.10/site-packages/torch/amp/autocast_mode.py:267: UserWarning: In CPU autocast, but the target dtype is not supported. Disabling autocast.
CPU Autocast only supports dtype of torch.bfloat16, torch.float16 currently.
  warnings.warn(error_message)
Batches: 100%|██████████| 6/6 [00:01<00:00,  3.46it/s]


Great! Our retriever is good to go. Lets load up an LLM, that will reason over the retrieved documents:

In [34]:
import torch
from langchain_huggingface.llms import HuggingFacePipeline
from optimum.intel.ipex import IPEXModelForCausalLM
from transformers import AutoTokenizer, pipeline

model_id = "Intel/neural-chat-7b-v3-3"
tokenizer = AutoTokenizer.from_pretrained(model_id)
model = IPEXModelForCausalLM.from_pretrained(
    model_id, torch_dtype=torch.bfloat16, export=True
)

pipe = pipeline("text-generation", model=model, tokenizer=tokenizer, max_new_tokens=300, min_new_tokens=300)

hf = HuggingFacePipeline(pipeline=pipe)

/home/rbrugaro/miniforge3/envs/llama-index/lib/python3.10/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
Detect torchscript is false. Convert to torchscript model!
Framework not specified. Using pt to export the model.


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Passing the argument `library_name` to `get_supported_tasks_for_model_type` is required, but got library_name=None. Defaulting to `transformers`. An error will be raised in a future version of Optimum if `library_name` is not provided.
/home/rbrugaro/miniforge3/envs/llama-index/lib/python3.10/site-packages/transformers/modeling_utils.py:4481: FutureWarning: `_is_quantized_training_enabled` is going to be deprecated in transformers 4.39.0. Please use `model.hf_quantizer.is_trainable` instead
  warnings.warn(
/home/rbrugaro/miniforge3/envs/llama-index/lib/python3.10/site-packages/transformers/modeling_attn_mask_utils.py:276: TracerWarning: Converting a tensor to a Python boolean might cause the trace to be incorrect. We can't record the data flow of Python values, so this value will be treated as a constant in the future. This means that the trace might not generalize to other inputs!
  elif sliding_window is None or key_value_length < sliding_window:
/home/rbrugaro/miniforge3/envs/llama

Next, we will load up a prompt for answering questions using retrieved documents:

In [35]:
from langchain import hub

prompt = hub.pull("rlm/rag-prompt")

In [36]:
print(prompt)

input_variables=['context', 'question'] metadata={'lc_hub_owner': 'rlm', 'lc_hub_repo': 'rag-prompt', 'lc_hub_commit_hash': '50442af133e61576e74536c6556cefe1fac147cad032f4377b60c436e6cdcb6e'} messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context', 'question'], template="You are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question. If you don't know the answer, just say that you don't know. Use three sentences maximum and keep the answer concise.\nQuestion: {question} \nContext: {context} \nAnswer:"))]


We can now build our pipeline:

In [37]:
from langchain.schema.runnable import RunnablePassthrough

rag_chain = {"context": retriever, "question": RunnablePassthrough()} | prompt | hf

Excellent! lets ask it a question.
We will also use a verbose and debug, to check which documents were used by the model to produce the answer.

In [38]:
langchain.verbose = True
langchain.debug = True

llm_res = rag_chain.invoke(
    "What is the first transduction model relying entirely on self-attention?",
)

[chain/start] [chain:RunnableSequence] Entering Chain run with input:
{
  "input": "What is the first transduction model relying entirely on self-attention?"
}
[chain/start] [chain:RunnableSequence > chain:RunnableParallel<context,question>] Entering Chain run with input:
{
  "input": "What is the first transduction model relying entirely on self-attention?"
}
[chain/start] [chain:RunnableSequence > chain:RunnableParallel<context,question> > chain:RunnablePassthrough] Entering Chain run with input:
{
  "input": "What is the first transduction model relying entirely on self-attention?"
}
[chain/end] [chain:RunnableSequence > chain:RunnableParallel<context,question> > chain:RunnablePassthrough] s] Exiting Chain run with output:
{
  "output": "What is the first transduction model relying entirely on self-attention?"
}


/home/rbrugaro/miniforge3/envs/llama-index/lib/python3.10/site-packages/torch/amp/autocast_mode.py:267: UserWarning: In CPU autocast, but the target dtype is not supported. Disabling autocast.
CPU Autocast only supports dtype of torch.bfloat16, torch.float16 currently.
  warnings.warn(error_message)
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
access to the `model_dtype` attribute is deprecated and will be removed after v1.18.0, please use `_dtype` instead.


[chain/end] [chain:RunnableSequence > chain:RunnableParallel<context,question>] s] Exiting Chain run with output:
[outputs]
[chain/start] [chain:RunnableSequence > prompt:ChatPromptTemplate] Entering Prompt run with input:
[inputs]
[chain/end] [chain:RunnableSequence > prompt:ChatPromptTemplate] s] Exiting Prompt run with output:
[outputs]
[llm/start] [chain:RunnableSequence > llm:HuggingFacePipeline] Entering LLM run with input:
{
  "prompts": [
    "Human: You are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question. If you don't know the answer, just say that you don't know. Use three sentences maximum and keep the answer concise.\nQuestion: What is the first transduction model relying entirely on self-attention? \nContext: [Document(metadata={'source': 'https://ar5iv.labs.arxiv.org/html/1706.03762', 'content_type': 'text/html; charset=utf-8', 'title': '[1706.03762] Attention Is All You Need', 'language': 'en'}, page_content

In [39]:
langchain.verbose = True
langchain.debug = True

llm_res = rag_chain.invoke(
    "What is the first transduction model relying entirely on self-attention?",
)

[chain/start] [chain:RunnableSequence] Entering Chain run with input:
{
  "input": "What is the first transduction model relying entirely on self-attention?"
}
[chain/start] [chain:RunnableSequence > chain:RunnableParallel<context,question>] Entering Chain run with input:
{
  "input": "What is the first transduction model relying entirely on self-attention?"
}
[chain/start] [chain:RunnableSequence > chain:RunnableParallel<context,question> > chain:RunnablePassthrough] Entering Chain run with input:
{
  "input": "What is the first transduction model relying entirely on self-attention?"
}
[chain/end] [chain:RunnableSequence > chain:RunnableParallel<context,question> > chain:RunnablePassthrough] s] Exiting Chain run with output:
{
  "output": "What is the first transduction model relying entirely on self-attention?"
}


/home/rbrugaro/miniforge3/envs/llama-index/lib/python3.10/site-packages/torch/amp/autocast_mode.py:267: UserWarning: In CPU autocast, but the target dtype is not supported. Disabling autocast.
CPU Autocast only supports dtype of torch.bfloat16, torch.float16 currently.
  warnings.warn(error_message)
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
access to the `model_dtype` attribute is deprecated and will be removed after v1.18.0, please use `_dtype` instead.


[chain/end] [chain:RunnableSequence > chain:RunnableParallel<context,question>] s] Exiting Chain run with output:
[outputs]
[chain/start] [chain:RunnableSequence > prompt:ChatPromptTemplate] Entering Prompt run with input:
[inputs]
[chain/end] [chain:RunnableSequence > prompt:ChatPromptTemplate] s] Exiting Prompt run with output:
[outputs]
[llm/start] [chain:RunnableSequence > llm:HuggingFacePipeline] Entering LLM run with input:
{
  "prompts": [
    "Human: You are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question. If you don't know the answer, just say that you don't know. Use three sentences maximum and keep the answer concise.\nQuestion: What is the first transduction model relying entirely on self-attention? \nContext: [Document(metadata={'source': 'https://ar5iv.labs.arxiv.org/html/1706.03762', 'content_type': 'text/html; charset=utf-8', 'title': '[1706.03762] Attention Is All You Need', 'language': 'en'}, page_content

Based on the retrieved documents, the answer is indeed correct :)